In [1]:
#Importing required packages.
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
#from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
rental_data = pd.read_csv("car_rental_large.csv",low_memory=False)
rental_data.head(10)

,Unnamed: 0,Booking Number,Price,Origin Lat,Origin Lon,Dest Lat,Dest Lon,Wait,Distance,Transport,Day,Date,Month,Year,Hour,Haul,Time Taken
0,0,AJ13943,204.07,28.429399,-81.308998,40.639801,-73.778900,180,2500,1,3,6,2,2019,1,0,188
1,1,AJ80110,854.84,45.470600,-73.740799,-33.946098,151.177002,210,100,0,4,24,12,2020,1,2,212
2,2,AJ65702,166.47,45.470600,-73.740799,40.639801,-73.778900,180,2500,1,2,24,11,2020,1,0,188
3,3,RY00938,2129.89,-33.946098,151.177002,49.193901,-123.183998,60,3400,0,2,6,8,2019,1,2,115
4,4,RY83421,374.55,-33.946098,151.177002,53.353699,-2.274950,180,1000,0,7,29,11,2020,1,2,196
5,5,VA09719,2348.47,45.470600,-73.740799,-33.946098,151.177002,390,100,0,6,11,1,2020,1,2,392
6,6,AA33777,848.60,51.470600,-0.461941,49.193901,-123.183998,60,3400,0,3,26,6,2019,1,2,116
7,7,AJ39289,1813.30,45.470600,-73.740799,-33.946098,151.177002,210,100,0,1,26,10,2020,1,2,212
8,8,AA72990,1558.65,51.470600,-0.461941,-33.946098,151.177002,390,100,0,6,5,12,2020,1,2,392
9,9,AA19087,826.64,40.639801,-73.778900,51.470600,-0.461941,240,800,1,5,25,12,2020,1,1,242


In [3]:
rental_data['Origin Lat'].value_counts()

 49.193901    25106
 51.470600    25095
 28.429399    25067
 53.630402    25058
 45.470600    25001
 53.353699    24961
 40.639801    24937
-33.946098    24775
Name: Origin Lat, dtype: int64

In [4]:
rental_data['Dest Lat'].value_counts()

 53.353699    25233
 49.193901    25121
-33.946098    25027
 45.470600    25006
 53.630402    24993
 40.639801    24912
 28.429399    24910
 51.470600    24798
Name: Dest Lat, dtype: int64

In [5]:
rental_data['Car Wait'] = rental_data['Wait'] - rental_data['Time Taken']

In [6]:
from math import isclose
def getID(x):
    if(isclose(x, 53.630402, abs_tol=0.001)):
        return 0
    if(isclose(x, 49.193901, abs_tol=0.001)):
        return 1
    if(isclose(x, -33.946098, abs_tol=0.001)):
        return 2
    if(isclose(x, 45.470600, abs_tol=0.001)):
        return 3
    if(isclose(x, 40.639801, abs_tol=0.001)):
        return 4
    if(isclose(x, 28.429399, abs_tol=0.001)):
        return 5
    if(isclose(x, 53.353699, abs_tol=0.001)):
        return 6
    if(isclose(x, 51.470600, abs_tol=0.001)):
        return 7
    return -1
rental_data['OriginID'] = rental_data['Origin Lat'].apply(getID)
rental_data['DestID'] = rental_data['Dest Lat'].apply(getID)

In [7]:
from math import sin, cos, sqrt, atan2, radians
for index, row in rental_data.iterrows():
    
    
    
    R = 6373.0

    lat1 = radians(row['Origin Lat'])
    lon1 = radians(row['Origin Lon'])
    lat2 = radians(row['Dest Lat'])
    lon2 = radians(row['Dest Lon'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    rental_data.at[index,'Flight Distance'] = distance

In [8]:
rental_data.head()

,Unnamed: 0,Booking Number,Price,Origin Lat,Origin Lon,Dest Lat,Dest Lon,Wait,Distance,Transport,...,Date,Month,Year,Hour,Haul,Time Taken,Car Wait,OriginID,DestID,Flight Distance
0,0,AJ13943,204.07,28.429399,-81.308998,40.639801,-73.778900,180,2500,1,...,6,2,2019,1,0,188,-8,5,4,1521.866215
1,1,AJ80110,854.84,45.470600,-73.740799,-33.946098,151.177002,210,100,0,...,24,12,2020,1,2,212,-2,3,2,16028.177679
2,2,AJ65702,166.47,45.470600,-73.740799,40.639801,-73.778900,180,2500,1,...,24,11,2020,1,0,188,-8,3,4,537.337885
3,3,RY00938,2129.89,-33.946098,151.177002,49.193901,-123.183998,60,3400,0,...,6,8,2019,1,2,115,-55,2,1,12504.876464
4,4,RY83421,374.55,-33.946098,151.177002,53.353699,-2.274950,180,1000,0,...,29,11,2020,1,2,196,-16,2,6,17017.658942


In [9]:
rental_data.head(10)

,Unnamed: 0,Booking Number,Price,Origin Lat,Origin Lon,Dest Lat,Dest Lon,Wait,Distance,Transport,...,Date,Month,Year,Hour,Haul,Time Taken,Car Wait,OriginID,DestID,Flight Distance
0,0,AJ13943,204.07,28.429399,-81.308998,40.639801,-73.778900,180,2500,1,...,6,2,2019,1,0,188,-8,5,4,1521.866215
1,1,AJ80110,854.84,45.470600,-73.740799,-33.946098,151.177002,210,100,0,...,24,12,2020,1,2,212,-2,3,2,16028.177679
2,2,AJ65702,166.47,45.470600,-73.740799,40.639801,-73.778900,180,2500,1,...,24,11,2020,1,0,188,-8,3,4,537.337885
3,3,RY00938,2129.89,-33.946098,151.177002,49.193901,-123.183998,60,3400,0,...,6,8,2019,1,2,115,-55,2,1,12504.876464
4,4,RY83421,374.55,-33.946098,151.177002,53.353699,-2.274950,180,1000,0,...,29,11,2020,1,2,196,-16,2,6,17017.658942
5,5,VA09719,2348.47,45.470600,-73.740799,-33.946098,151.177002,390,100,0,...,11,1,2020,1,2,392,-2,3,2,16028.177679
6,6,AA33777,848.60,51.470600,-0.461941,49.193901,-123.183998,60,3400,0,...,26,6,2019,1,2,116,-56,7,1,7580.963373
7,7,AJ39289,1813.30,45.470600,-73.740799,-33.946098,151.177002,210,100,0,...,26,10,2020,1,2,212,-2,3,2,16028.177679
8,8,AA72990,1558.65,51.470600,-0.461941,-33.946098,151.177002,390,100,0,...,5,12,2020,1,2,392,-2,7,2,17025.997057
9,9,AA19087,826.64,40.639801,-73.778900,51.470600,-0.461941,240,800,1,...,25,12,2020,1,1,242,-2,4,7,5541.383076


In [10]:
X = rental_data[['DestID','Day','Transport','Flight Distance']]
y = rental_data['Car Wait']

In [11]:
X.head()

,DestID,Day,Transport,Flight Distance
0,4,3,1,1521.866215
1,2,4,0,16028.177679
2,4,2,1,537.337885
3,1,2,0,12504.876464
4,6,7,0,17017.658942


In [12]:
y.head()

0    -8
1    -2
2    -8
3   -55
4   -16
Name: Car Wait, dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [14]:
print(y)

0         -8
1         -2
2         -8
3        -55
4        -16
          ..
199995   -14
199996    -8
199997    -4
199998    -2
199999    -2
Name: Car Wait, Length: 200000, dtype: int64


In [15]:
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.fit_transform(X_test)

In [16]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

In [17]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

         -56       0.00      0.00      0.00       132
         -55       0.20      0.19      0.19       287
         -54       0.25      0.22      0.23       269
         -53       0.20      0.21      0.20       264
         -52       0.20      0.41      0.26       256
         -51       0.00      0.00      0.00       104
         -50       0.00      0.00      0.00        90
         -49       0.23      0.11      0.15       206
         -48       0.18      0.51      0.27       172
         -47       0.26      0.14      0.18       196
         -46       0.18      0.27      0.22       195
         -45       0.00      0.00      0.00       144
         -44       0.00      0.00      0.00        72
         -43       0.00      0.00      0.00        60
         -42       0.32      0.86      0.47       100
         -41       0.24      0.06      0.10       197
         -40       0.00      0.00      0.00       116
         -39       0.30    

C:\Users\Nick\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
print(confusion_matrix(y_test, pred_rfc))

[[   0   29   21 ...    0    0    0]
 [   0   54   43 ...    0    0    0]
 [   0   57   59 ...    0    0    0]
 ...
 [   0    0    0 ... 2525    0    0]
 [   0    0    0 ...    0 7030    0]
 [   0    0    0 ...    0    0  603]]


In [19]:
from sklearn.metrics import accuracy_score
cm = accuracy_score(y_test, pred_rfc)

cm

0.780875

In [20]:
X.head()

,DestID,Day,Transport,Flight Distance
0,4,3,1,1521.866215
1,2,4,0,16028.177679
2,4,2,1,537.337885
3,1,2,0,12504.876464
4,6,7,0,17017.658942


In [21]:
y.head()

0    -8
1    -2
2    -8
3   -55
4   -16
Name: Car Wait, dtype: int64

In [22]:
#Xnew = [[7,5,1,5,800,0]]
#ynew = rfc.predict(Xnew)
#print(ynew)

In [23]:
X_test.head()

,DestID,Day,Transport,Flight Distance
119737,0,3,1,811.086195
72272,2,2,0,17017.658942
158154,7,5,0,745.677405
65426,2,6,0,0.000000
30074,7,4,1,5541.383076


In [24]:
y_test.head()

119737    -4
72272     -2
158154   -10
65426     -1
30074     -2
Name: Car Wait, dtype: int64

In [25]:
temp_DestID = list(X_test['DestID'])
temp_Day = list(X_test['Day'])
temp_Transport = list(X_test['Transport'])
temp_Flight_Distance = list(X_test['Flight Distance'])

temp_y = list(y_test)
differences = 0
offby=[0,0,0,0,0,0]

for i in range(len(y_test)):
    temp = rfc.predict([[temp_DestID[i],temp_Day[i],temp_Transport[i],temp_Flight_Distance[i]]])
    off = abs((temp-temp_y[i]))
    differences += off
    if(off==0):
        offby[0]+=1
    elif(off==1):
        offby[1]+=1
    elif(off==2):
        offby[2]+=1
    elif(off<=5):
        offby[3]+=1
    elif(off<=10):
        offby[4]+=1
    else:
        offby[5]+=1

In [26]:
print(differences/len(y_test))

[0.300475]


In [27]:
print(offby)

[31235, 6380, 1634, 751, 0, 0]


In [28]:
count = 0
for i in offby:
    print("Off by %d: %f%%"% (count, i/len(y_test)*100))
    count+=1

Off by 0: 78.087500%
Off by 1: 15.950000%
Off by 2: 4.085000%
Off by 3: 1.877500%
Off by 4: 0.000000%
Off by 5: 0.000000%


In [29]:
for i in range(1,20,1):
    rfc = RandomForestClassifier(n_estimators=i)
    rfc.fit(X_train, y_train)
    pred_rfc = rfc.predict(X_test)
    print(accuracy_score(y_test, pred_rfc))

0.781475
0.7814
0.7816
0.78055
0.78095
0.780975
0.781125
0.78075
0.779675
0.78195
0.781125
0.781025
0.780525
0.781025
0.78215
0.781875
0.7814
0.781425
0.78105
